In [1]:
#if your libraries are not loading, uncomment the relevant line to install

#Random
# import Pkg; Pkg.add("ACME")

#DataFrames
# import Pkg; Pkg.add("DataFrames")

#JuMP
# import Pkg; Pkg.add("JuMP")

#GLPKMathProgInterface
# import Pkg; Pkg.add("GLPKMathProgInterface")

In [2]:
#load all the libraries and functions we will need
using Random
using DataFrames
using JuMP, GLPKMathProgInterface
using Base

function GenerateExampleBusData(number_of_buses, maxHoursToRepair, minHoursToRepair, failuresPerBus, maxDays, partList = ["battery","tires","brakes","air filter","engine cooling","transmission"])
    #make buses
    bus_df = DataFrame(
        bus_id = string.("bus_",collect(1:number_of_buses))
        , shop_id="shop_1")

    #make the parts
    part_type_df = DataFrame(
        part_id = string.("part_",collect(1:length(partList)))
        , part_type_name = partList
        , hours_to_repair = rand(minHoursToRepair:maxHoursToRepair, length(partList)))

    #make the failures
    pred_failure_df = DataFrame(
        pred_failure_id = string.("pred_",collect(1:(number_of_buses * failuresPerBus)))
        , bus_id = rand(bus_df[:,Symbol("bus_id")], (number_of_buses * failuresPerBus))
        , part_id = rand(part_type_df[:,Symbol("part_id")], (number_of_buses * failuresPerBus))
        , days_to_failure = rand(1:maxDays, number_of_buses * failuresPerBus))

    print("data generation complete \n")
    return [bus_df, part_type_df, pred_failure_df]
end

GenerateExampleBusData (generic function with 2 methods)

# STAGE 1

## SUMMARY

### Variables:
Xij: binary. Whether a given truck is in a shop on a given day.  Total number of X is the number of buses (i) multiplied by the number of days the model considers (j).  In the code it takes on a matrix of the dimensions i by j

### Objective:
Min sum(Xij).  The stage 1 object is straightforward.  Minimize the number days a bus spends in a shop

### Stage Considerations:
* This stage only works because we assume that all teh issues with a single bus can be fixed within a single shop visit.  This allows the constraint pertaining to the part failure to be simple: bring it in before the earliest part failure on the bus.

## PROBLEM NOTATION

$variable: X_{ij}$, boolean, (bus_in_shop_on_day)

$constraint: \sum_{i=1:I} X_{ij} \leq SB \space \forall j$ (only_so_much_space)

$constraint: \sum_{j=1:min(TTF \space for \space all \space k \space on \space i)} X_{ij} \geq 1 \space \forall i$ (bring_it_in_and_fix_it_all_at_once)

$objective: min(\sum_{i=1} \sum_{j=1} X_{ij})$


In [3]:
################################################
############# set up the data ##################
################################################
Random.seed!(1234);

#bus company specifics
number_of_buses = 5
maxHoursToRepair = 8
minHoursToRepair = 1
failuresPerBus = 3
maxDays = 14

#generate bus company data
dataList = GenerateExampleBusData(number_of_buses, maxHoursToRepair, minHoursToRepair, failuresPerBus, maxDays)
bus_df = dataList[1]
part_type_df = dataList[2]
pred_failure_df = dataList[3]

#stage one constants
shopBerths = 3

###################################################
############# set up the program ##################
###################################################

#make model
busScheduleModel = Model(solver=GLPKSolverMIP())

#make the variables
@variable(busScheduleModel, 0 <= X[1:nrow(bus_df),1:maxDays] <= 1, Int)

#make the gotta_save_the_bus constraint
#TODO: make the max of the j function that finds the min failure date of a specific part
for i in 1:nrow(bus_df)
    if length(findall(pred_failure_df.bus_id .== bus_df.bus_id[i])) > 0
        daysOutMin = minimum(pred_failure_df[findall(pred_failure_df.bus_id .== bus_df.bus_id[i]),Symbol("days_to_failure")])
        @constraint(busScheduleModel, sum{X[i,j],j=1:daysOutMin} >= 1)
    end
end

#make the only_so_much_space constraint
for j in 1:maxDays
    @constraint(busScheduleModel, sum{X[i,j],i=1:nrow(bus_df)} <= shopBerths)
end

#make the objective function
@objective(busScheduleModel,Min,sum{X[i,j],i=1:nrow(bus_df),j=1:maxDays})

#print out what it looks like
print(busScheduleModel)

#solve it i guess
@time begin
    status = solve(busScheduleModel)
end

data generation complete 
Min X[1,1] + X[1,2] + X[1,3] + X[1,4] + X[1,5] + X[1,6] + X[1,7] + X[1,8] + X[1,9] + X[1,10] + X[1,11] + X[1,12] + X[1,13] + X[1,14] + X[2,1] + X[2,2] + X[2,3] + X[2,4] + X[2,5] + X[2,6] + X[2,7] + X[2,8] + X[2,9] + X[2,10] + X[2,11] + X[2,12] + X[2,13] + X[2,14] + X[3,1] + X[3,2] + X[3,3] + X[3,4] + X[3,5] + X[3,6] + X[3,7] + X[3,8] + X[3,9] + X[3,10] + X[3,11] + X[3,12] + X[3,13] + X[3,14] + X[4,1] + X[4,2] + X[4,3] + X[4,4] + X[4,5] + X[4,6] + X[4,7] + X[4,8] + X[4,9] + X[4,10] + X[4,11] + X[4,12] + X[4,13] + X[4,14] + X[5,1] + X[5,2] + X[5,3] + X[5,4] + X[5,5] + X[5,6] + X[5,7] + X[5,8] + X[5,9] + X[5,10] + X[5,11] + X[5,12] + X[5,13] + X[5,14]
Subject to
 X[1,1] ≥ 1
 X[2,1] + X[2,2] + X[2,3] + X[2,4] + X[2,5] + X[2,6] ≥ 1
 X[3,1] + X[3,2] + X[3,3] + X[3,4] + X[3,5] + X[3,6] + X[3,7] ≥ 1
 X[4,1] + X[4,2] + X[4,3] + X[4,4] + X[4,5] + X[4,6] ≥ 1
 X[5,1] + X[5,2] + X[5,3] ≥ 1
 X[1,1] + X[2,1] + X[3,1] + X[4,1] + X[5,1] ≤ 3
 X[1,2] + X[2,2] + X[3,2] + X[4,2] +

:Optimal

### Solved objective:

In [4]:
println("Bus days in shop (objective): ", getobjectivevalue(busScheduleModel))

Bus days in shop (objective): 5.0


### Solved X variables:
All the buses come into the shop as soon as possible.

In [5]:
getvalue(X)

5×14 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

# STAGE 2
## SUMMARY

### Variables:
Xij: binary. Whether a given truck is in a shop on a given day.  Total number of X is the number of buses (i) multiplied by the number of days the model considers (j).  In the code it takes on a matrix of the dimensions i by j

Yij: How many hours are spent on a bus when it comes into the shop. Same dimensions and number of variables as X.  Allows additional constraints to be put into place around how much work can be done in a day.

### Objective:
Min sum(Xij).  Same as stage 2. But in this stage the additional constraints may force more shop visits and more complicated optimization than bring everything in as soon as possible.

### Stage Considerations:
* The gotta_save_the_part constraint is what allows for additive part repairs. It assumes that the most urgent failing parts will be fixed first. If there are two part repairs on a given bus, the time spent on the bus by the time of the second part failure must be at least the sum of the time to fix both failures.
* The gotta_be_in_the_shop_to_work_on constraint is what links Y to X and thus to the objective function. Since X is binary, diving Y by the number of shop_hours and requiring X to be greater than that, will toggle the given bus to in the shop whenever it work will be done on it.

## PROBLEM NOTATION

$variable: X_{ij}$, boolean, (bus_in_shop_on_day)

$variable: Y_{ij} \geq 0$ cont, (hours_spent_on_bus_on_day) 

$constraint: \sum_{i=1:I} X_{ij} \leq SB \space \forall j$ (only_so_much_space)

$constraint: \sum_{i=1:I} Y_{ij} \leq SH \space \forall j$ (only_so_much_time)

$constraint: X_{ij} \geq Y_{ij}/(SH) \space \forall i \forall j$ (gotta_be_in_the_shop_to_work_on)

$constraint: \sum_{i=(i \space of \space k)} \sum_{j=1:(TTF \space of \space k)} Y_{ij} \geq \sum_{k=K_i} TOR_k \space \forall k$ (gotta_save_the_part)

$objective: min(\sum_{i=1} \sum_{j=1} X_{ij})$

In [6]:
################################################
############# set up the data ##################
################################################
Random.seed!(1234);

#bus company specifics
number_of_buses = 5
maxHoursToRepair = 8
minHoursToRepair = 1
failuresPerBus = 3
maxDays = 14

#generate bus company data
dataList = GenerateExampleBusData(number_of_buses, maxHoursToRepair, minHoursToRepair, failuresPerBus, maxDays)
bus_df = dataList[1]
part_type_df = dataList[2]
pred_failure_df = dataList[3]

#stage one constants
shopBerths = 3

#stage two constants
shopHours = 16

###################################################
############# set up the program ##################
###################################################

#make model
busScheduleModel = Model(solver=GLPKSolverMIP())

#make the variables
@variable(busScheduleModel, 0 <= X[1:nrow(bus_df),1:maxDays] <= 1, Int)
@variable(busScheduleModel, Y[1:nrow(bus_df),1:maxDays] >= 0)

print("made the variables \n")

#make the gotta_save_the_part(s) constraint
for big_k in 1:nrow(pred_failure_df)
    #find the days we have to solve this constraint
    days_to_failure = pred_failure_df[big_k,Symbol("days_to_failure")]
    #print(days_to_failure)
    #print("\n")
    
    #find the relevant i (bus index) to associate with this constraint
    i = findall(bus_df.bus_id .== pred_failure_df[big_k,Symbol("bus_id")])[1,1]
    #print(i)
    #print("\n")
    
    #find all little_k with time to fail less than or equal big_k, sum their repair times
    index_of_failure_predictions = intersect(findall(pred_failure_df.bus_id .== pred_failure_df[big_k,Symbol("bus_id")])
                                                               ,findall(pred_failure_df.days_to_failure .<= pred_failure_df[big_k,Symbol("days_to_failure")]))
    failures_preds_with_ttrepair_df = join(pred_failure_df, part_type_df, on = :part_id, kind = :left)
    hours_of_repair_for_this_k = sum(failures_preds_with_ttrepair_df[index_of_failure_predictions,Symbol("hours_to_repair")])
    #print(hours_of_repair_for_this_k)
    #print("\n")
    
    #make the constraint
    @constraint(busScheduleModel, sum{Y[i,j],j=1:days_to_failure} >= hours_of_repair_for_this_k)
end

print("made the gotta_save_the_part constraint \n")

#make the if gotta_be_in_the_shop_to_work_on constraint
for i in 1:nrow(bus_df)
    for j in 1:maxDays
        @constraint(busScheduleModel, X[i,j] >= (Y[i,j]/(shopHours + 1)))
    end
end

print("made the gotta_be_in_the_shop_to_work_on constraint \n")

#make the only_so_much_space constraint
for j in 1:maxDays
    @constraint(busScheduleModel, sum{X[i,j],i=1:nrow(bus_df)} <= shopBerths)
end

print("made the only_so_much_space constraint \n")

#make the only_so_much_time constraint
for j in 1:maxDays
    @constraint(busScheduleModel, sum{Y[i,j],i=1:nrow(bus_df)} <= shopHours)
end

print("made the only_so_much_time constraint \n")

#make the objective function
@objective(busScheduleModel,Min,sum{X[i,j],i=1:nrow(bus_df),j=1:maxDays})

#print out what it looks like
print(busScheduleModel)

#solve it i guess
@time begin
    status = solve(busScheduleModel)
end

data generation complete 
made the variables 
made the gotta_save_the_part constraint 
made the gotta_be_in_the_shop_to_work_on constraint 
made the only_so_much_space constraint 
made the only_so_much_time constraint 
Min X[1,1] + X[1,2] + X[1,3] + X[1,4] + X[1,5] + X[1,6] + X[1,7] + X[1,8] + X[1,9] + X[1,10] + X[1,11] + X[1,12] + X[1,13] + X[1,14] + X[2,1] + X[2,2] + X[2,3] + X[2,4] + X[2,5] + X[2,6] + X[2,7] + X[2,8] + X[2,9] + X[2,10] + X[2,11] + X[2,12] + X[2,13] + X[2,14] + X[3,1] + X[3,2] + X[3,3] + X[3,4] + X[3,5] + X[3,6] + X[3,7] + X[3,8] + X[3,9] + X[3,10] + X[3,11] + X[3,12] + X[3,13] + X[3,14] + X[4,1] + X[4,2] + X[4,3] + X[4,4] + X[4,5] + X[4,6] + X[4,7] + X[4,8] + X[4,9] + X[4,10] + X[4,11] + X[4,12] + X[4,13] + X[4,14] + X[5,1] + X[5,2] + X[5,3] + X[5,4] + X[5,5] + X[5,6] + X[5,7] + X[5,8] + X[5,9] + X[5,10] + X[5,11] + X[5,12] + X[5,13] + X[5,14]
Subject to
 Y[5,1] + Y[5,2] + Y[5,3] + Y[5,4] ≥ 15
 Y[4,1] + Y[4,2] + Y[4,3] + Y[4,4] + Y[4,5] + Y[4,6] ≥ 4
 Y[3,1] + Y[3,2]

:Optimal

### Solved Objective:

In [7]:
println("Bus days in shop (objective): ", getobjectivevalue(busScheduleModel))

Bus days in shop (objective): 9.0


### Solved X variable:
With the same number of buses and the same failures, the constraint of time forces he buses to come in more othen.  The shop visits are now spread more evenly over all the days.

In [8]:
getvalue(X)

5×14 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

### Solved Y variable:

All the shop days are the beginning of the rage are filling up on 16 hours shifts (the max).  Toward the end no work is done since much of the more evenly spread part failures were fixed in the front-loaded shop hours.

In [9]:
getvalue(Y)

5×14 Array{Float64,2}:
 16.0   0.0   0.0   0.0          0.0  …  0.0   0.0  0.0  0.0   0.0  0.0  0.0
  0.0   0.0  10.0  -9.4369e-16   0.0     0.0  14.0  0.0  0.0   0.0  0.0  0.0
  0.0   0.0   0.0  16.0          0.0     0.0   0.0  0.0  0.0  13.0  0.0  0.0
  0.0   0.0   0.0   0.0         16.0     0.0   2.0  0.0  0.0   0.0  0.0  0.0
  0.0  16.0   6.0   0.0          0.0     0.0   0.0  0.0  0.0   0.0  0.0  0.0

# STAGE 3

## SUMMARY 

### Variables:
Xij: binary. Whether a given truck is in a shop on a given day.  Total number of X is the number of buses (i) multiplied by the number of days the model considers (j).  In the code it takes on a matrix of the dimensions i by j

Yij: How many hours are spent on a bus when it comes into the shop. Same dimensions and number of variables as X.  Allows additional constraints to be put into place around how much work can be done in a day.

SH: The fixed shop hours in the previous stage have become variable. This vairable is used in conjunction with the new financial objective.  Unlike the other variables this is a single number rather than matrix.  A more complex implementation could involve some sort of schedule.

### Objective:
Minimize cost! Two new constants are defined: cost_per_shop_hour (CsH) and cost_per_shop_day (CsD).  The cost to operate the shop is the shop hours multiplied by the cost per hour multiplied by the number of days.  An aditional large penalty for bringing the bus into the shop is factored into the optimization to ensure that the number of shop visits remains low.

### Stage Considerations:
* Despite being very simular to stage two, the addition of a thrid variable will overload the solver undercertain conditions.  Play around with the random seed and you will arrive at dramatically different results.  With under 20 variables it is possilbe to solve this problem in seconds although sometimes it appears it does get stuck.  Seed 1234 will not solve, seed 333 solves in less than a second, 555 takes around 10 seconds.
* The shop hours can no longer be used in the gotta_be_in_the_shop_to_work_on constaint or else it would become non-linear. As such a very large number has been subsituted and achieves the same effect.  The problem could break if the shop hours exceeds this number but that is very unlikely.
* While the next stage remains unexplored, this could be the stepping point to simulating lots of part breakdown schenarios and finding the optimal staffing strategy within a certain confidence and price range.

## PROBLEM NOTATION

$variable: X_{ij}$, boolean, (bus_in_shop_on_day)

$variable: Y_{ij} \geq 0$, cont, (hours_spent_on_bus_on_day)

$variable: SH \geq 0$, cont, (hours_of_shop_labor_every_day)

$constraint: \sum_{i=1:I} X_{ij} \leq SB \space \forall j$ (only_so_much_space)

$constraint: \sum_{i=1:I} Y_{ij} \leq SH \space \forall j$ (only_so_much_time)

$constraint: X_{ij} \geq Y_{ij}/(1000000) \space \forall i \forall j$ (gotta_be_in_the_shop_to_work_on)

$constraint: \sum_{i=(i \space of \space k)} \sum_{j=1:(TTF \space of \space k)} Y_{ij} \geq \sum_{k=K_i} TOR_k \space \forall k$ (gotta_save_the_part)

$objective: min((\sum_{i=1:I} \sum_{j=1:J} X_{ij})*CsD \space + \space J*CsH*SH)$ (people_cost_money_but_busses_cost_more)

In [23]:
################################################
############# set up the data ##################
################################################
Random.seed!(555);

#bus company specifics
number_of_buses = 5
maxHoursToRepair = 8
minHoursToRepair = 1
failuresPerBus = 3
maxDays = 10

#generate bus company data
dataList = GenerateExampleBusData(number_of_buses, maxHoursToRepair, minHoursToRepair, failuresPerBus, maxDays)
bus_df = dataList[1]
part_type_df = dataList[2]
pred_failure_df = dataList[3]

#stage one constants
shopBerths = 2

#stage two constants (now a variable)
#shopHours = 16

#stage 3 constants
CsH = 20
CsD = 1000

###################################################
############# set up the program ##################
###################################################

#make model
busScheduleModel = Model(solver=GLPKSolverMIP())

#make the variables
@variable(busScheduleModel, 0 <= X[1:nrow(bus_df),1:maxDays] <= 1, Int)
@variable(busScheduleModel, Y[1:nrow(bus_df),1:maxDays] >= 0)
@variable(busScheduleModel, SH >= 0)

print("made the variables \n")

#make the gotta_save_the_part(s) constraint
for big_k in 1:nrow(pred_failure_df)
    #find the days we have to solve this constraint
    days_to_failure = pred_failure_df[big_k,Symbol("days_to_failure")]
    #print(days_to_failure)
    #print("\n")
    
    #find the relevant i (bus index) to associate with this constraint
    i = findall(bus_df.bus_id .== pred_failure_df[big_k,Symbol("bus_id")])[1,1]
    #print(i)
    #print("\n")
    
    #find all little_k with time to fail less than or equal big_k, sum their repair times
    index_of_failure_predictions = intersect(findall(pred_failure_df.bus_id .== pred_failure_df[big_k,Symbol("bus_id")])
                                                               ,findall(pred_failure_df.days_to_failure .<= pred_failure_df[big_k,Symbol("days_to_failure")]))
    failures_preds_with_ttrepair_df = join(pred_failure_df, part_type_df, on = :part_id, kind = :left)
    hours_of_repair_for_this_k = sum(failures_preds_with_ttrepair_df[index_of_failure_predictions,Symbol("hours_to_repair")])
    #print(hours_of_repair_for_this_k)
    #print("\n")
    
    #make the constraint
    @constraint(busScheduleModel, sum{Y[i,j],j=1:days_to_failure} >= hours_of_repair_for_this_k)
end

print("made the gotta_save_the_part constraint \n")

#make the if gotta_be_in_the_shop_to_work_on constraint
for i in 1:nrow(bus_df)
    for j in 1:maxDays
        @constraint(busScheduleModel, X[i,j] >= (Y[i,j]/(100 + 1)))
    end
end

print("made the gotta_be_in_the_shop_to_work_on constraint \n")

#make the only_so_much_space constraint
for j in 1:maxDays
    @constraint(busScheduleModel, sum{X[i,j],i=1:nrow(bus_df)} <= shopBerths)
end

print("made the only_so_much_space constraint \n")

#make the only_so_much_time constraint
for j in 1:maxDays
    @constraint(busScheduleModel, sum{Y[i,j],i=1:nrow(bus_df)} <= SH)
end

print("made the only_so_much_time constraint \n")

#make the objective function
@objective(busScheduleModel,Min,sum{X[i,j] * CsD,i=1:nrow(bus_df),j=1:maxDays} + CsH*SH*maxDays)

#print out what it looks like
print(busScheduleModel)

#solve it i guess
@time begin
    status = solve(busScheduleModel)
end

data generation complete 
made the variables 
made the gotta_save_the_part constraint 
made the gotta_be_in_the_shop_to_work_on constraint 
made the only_so_much_space constraint 
made the only_so_much_time constraint 
Min 1000 X[1,1] + 1000 X[1,2] + 1000 X[1,3] + 1000 X[1,4] + 1000 X[1,5] + 1000 X[1,6] + 1000 X[1,7] + 1000 X[1,8] + 1000 X[1,9] + 1000 X[1,10] + 1000 X[2,1] + 1000 X[2,2] + 1000 X[2,3] + 1000 X[2,4] + 1000 X[2,5] + 1000 X[2,6] + 1000 X[2,7] + 1000 X[2,8] + 1000 X[2,9] + 1000 X[2,10] + 1000 X[3,1] + 1000 X[3,2] + 1000 X[3,3] + 1000 X[3,4] + 1000 X[3,5] + 1000 X[3,6] + 1000 X[3,7] + 1000 X[3,8] + 1000 X[3,9] + 1000 X[3,10] + 1000 X[4,1] + 1000 X[4,2] + 1000 X[4,3] + 1000 X[4,4] + 1000 X[4,5] + 1000 X[4,6] + 1000 X[4,7] + 1000 X[4,8] + 1000 X[4,9] + 1000 X[4,10] + 1000 X[5,1] + 1000 X[5,2] + 1000 X[5,3] + 1000 X[5,4] + 1000 X[5,5] + 1000 X[5,6] + 1000 X[5,7] + 1000 X[5,8] + 1000 X[5,9] + 1000 X[5,10] + 200 SH
Subject to
 Y[4,1] + Y[4,2] + Y[4,3] ≥ 13
 Y[5,1] + Y[5,2] + Y[5,

:Optimal

### Solved objective and SH variable:

In [24]:
println("The cost of operating the shop in this scenario: ", getobjectivevalue(busScheduleModel))
println("The number of man hours required per day: ", getvalue(SH))

The cost of operating the shop in this scenario: 10466.666666666666
The number of man hours required per day: 17.333333333333332


### Solved X variable:

In [25]:
getvalue(X)

5×10 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0

### Solved Y variable:

In [26]:
getvalue(Y)

5×10 Array{Float64,2}:
  0.0      0.0       4.0      0.0      0.0     0.0  0.0  0.0  0.0  0.0
  0.0      5.66667   0.0     17.3333   0.0     0.0  0.0  0.0  0.0  0.0
 17.3333   0.0       0.0      0.0      0.0     0.0  0.0  0.0  0.0  0.0
  0.0      0.0      13.3333   0.0      0.0     0.0  0.0  0.0  0.0  0.0
  0.0     11.6667    0.0      0.0     17.3333  0.0  0.0  0.0  0.0  0.0